In [0]:
%sql
CREATE TABLE table1;
CREATE TABLE table2;
/*
Tạo bảng theo kiểu default như trên, metadata của bảng table1 và table2 sẽ được lưu ở dbfs:/user/hive/warehouse
*/

In [0]:
%sql
CREATE SCHEMA db_x;

In [0]:
%sql
use db_x;
create table table1;
create table table2;
/*
Tạo schema riêng và metadata của bảng sẽ được lưu ở trong hive metastore
*/

In [0]:
%fs
ls dbfs:/Workspace